In [1]:
import json
from modules.training_module import TrainingModule
from modules.model_evaluator import ModelEvaluator
from configs.base_config import TrainingModuleConfig
from configs.base_config import ModelEvaluatorConfig

In [12]:
sample_train_config = json.loads('{ "region_name": "Bengaluru", "train_start_date": "4/13/20", "train_end_date": "4/20/20", "model_class": "SEIR", "model_parameters": { "infectious_period": 9.5412, "incubation_period": 6, "IbyCRatio": 1 }, "search_space": { "r0": [ 0.8, 6 ], "EbyCRatio": [ 1, 70 ] }, "search_parameters": { "max_evals": 10 }, "loss_functions": [ { "metric_name": "mape", "variable_weights": [ { "variable": "active", "weight": 1 } ], "optimize": true}, { "metric_name": "rmse", "variable_weights": [ { "variable": "active", "weight": 1 } ] } ], "output_filepath": "seir_train.json" }')
sample_test_config = json.loads('{ "region_name": "Bengaluru", "run_day": "4/20/20", "test_start_date": "4/21/20", "test_end_date": "4/27/20", "model_class": "SEIR", "model_parameters": {"infectious_period": 9.5412, "r0": 2.4091, "incubation_period": 6, "IbyCRatio": 1}, "loss_functions": [ { "metric_name": "mape", "variable_weights": [ { "variable": "active", "weight": 1 } ], "optimize": true }, { "metric_name": "rmse", "variable_weights": [ { "variable": "active", "weight": 1 } ] } ], "output_filepath": "seir_evaluation_output.json" }')
sample_finaltrain_config = json.loads('{ "region_name": "Bengaluru", "train_start_date": "4/21/20", "train_end_date": "4/27/20", "model_class": "SEIR", "model_parameters": { "infectious_period": 9.5412, "incubation_period": 6, "IbyCRatio": 1 }, "search_space": { "r0": [ 0.8, 6 ], "EbyCRatio": [ 1, 70 ] }, "search_parameters": { "max_evals": 10 }, "loss_functions": [ { "metric_name": "mape", "variable_weights": [ { "variable": "active", "weight": 1 } ], "optimize": true}, { "metric_name": "rmse", "variable_weights": [ { "variable": "active", "weight": 1 } ] } ], "output_filepath": "seir_train.json" }')

#runday will be startday-1

In [13]:
sample_train_config

{'region_name': 'Bengaluru',
 'train_start_date': '4/13/20',
 'train_end_date': '4/20/20',
 'model_class': 'SEIR',
 'model_parameters': {'infectious_period': 9.5412,
  'incubation_period': 6,
  'IbyCRatio': 1},
 'search_space': {'r0': [0.8, 6], 'EbyCRatio': [1, 70]},
 'search_parameters': {'max_evals': 10},
 'loss_functions': [{'metric_name': 'mape',
   'variable_weights': [{'variable': 'active', 'weight': 1}],
   'optimize': True},
  {'metric_name': 'rmse',
   'variable_weights': [{'variable': 'active', 'weight': 1}]}],
 'output_filepath': 'seir_train.json'}

In [14]:
sample_test_config

{'region_name': 'Bengaluru',
 'run_day': '4/20/20',
 'test_start_date': '4/21/20',
 'test_end_date': '4/27/20',
 'model_class': 'SEIR',
 'model_parameters': {'infectious_period': 9.5412,
  'r0': 2.4091,
  'incubation_period': 6,
  'IbyCRatio': 1},
 'loss_functions': [{'metric_name': 'mape',
   'variable_weights': [{'variable': 'active', 'weight': 1}],
   'optimize': True},
  {'metric_name': 'rmse',
   'variable_weights': [{'variable': 'active', 'weight': 1}]}],
 'output_filepath': 'seir_evaluation_output.json'}

In [15]:
sample_finaltrain_config

{'region_name': 'Bengaluru',
 'train_start_date': '4/21/20',
 'train_end_date': '4/27/20',
 'model_class': 'SEIR',
 'model_parameters': {'infectious_period': 9.5412,
  'incubation_period': 6,
  'IbyCRatio': 1},
 'search_space': {'r0': [0.8, 6], 'EbyCRatio': [1, 70]},
 'search_parameters': {'max_evals': 10},
 'loss_functions': [{'metric_name': 'mape',
   'variable_weights': [{'variable': 'active', 'weight': 1}],
   'optimize': True},
  {'metric_name': 'rmse',
   'variable_weights': [{'variable': 'active', 'weight': 1}]}],
 'output_filepath': 'seir_train.json'}

In [16]:
regions = [('Bengaluru', 'district')]#, ('Pune', 'district'), ('Karnataka', 'state'), ('Rajasthan', 'state')]

resultDict = dict()
for region in regions:
    resultDict[region[0]] = dict()
   
    train_config = sample_train_config.copy()
    train_config['region_name'] = region[0]
    train_config['region_type'] = region[1]
    train_module_config = TrainingModuleConfig.parse_obj(train_config)
    trainResults = TrainingModule.from_config(train_module_config)
    
    print("Training Complete")
    
    #TODO: Might need to change IbyCRatio if the seed data is available for April 20th
    
    test_config = sample_test_config.copy()
    test_config['region_name'] = region[0]
    test_config['region_type'] = region[1]
    test_config['model_parameters'].update(trainResults['best_params'])
    test_config['model_parameters'].update(trainResults['latent_params'])
    test_module_config = ModelEvaluatorConfig.parse_obj(test_config) 
    evalResults = ModelEvaluator.from_config(test_module_config)
    print(evalResults)
    
    print("Evaluation Complete")
    finalTrain_config = sample_finaltrain_config.copy()
    
    #TODO: Might need to change IbyCRatio if the seed data is available for April 20th
    
    
    finalTrain_config['region_name'] = region[0]
    finalTrain_config['region_type'] = region[1]
    finalTrain_module_config = TrainingModuleConfig.parse_obj(finalTrain_config)
    finalResults = TrainingModule.from_config(finalTrain_module_config)
    
    #TODO: Might need to change IbyCRatio if the seed data is available for April 27th before exporting final params
    
    

t = 7.90                                              
t = 7.90                                              
t = 7.90                                                                        
t = 7.90                                                                        
t = 7.90                                                                        
t = 7.90                                                                       
t = 7.90                                                                       
t = 7.90                                                                       
t = 7.90                                                                       
t = 7.90                                                                       
100%|██████████| 10/10 [00:00<00:00, 17.87trial/s, best loss: 851.3313101522183]
Best fit: {'EbyCRatio': 18.58459141637886, 'r0': 5.628594639769091}
Best Loss: 851.3313101522183
t = 7.90
t = 7.90
Training Complete
t = 7.90
[{'metric_name': <Metric